# Final Solution

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from xgboost import XGBoostClassifier
from catboost import CatBoostClassifier

In [ ]:
train = pd.read_csv('scaled_data/train_scaled.csv', index_col=0)
test = pd.read_csv('scaled_data/test_scaled.csv', index_col=0)

X_train = train.drop(['id', 'Response'], axis = 1)
y_train = train['Response'].values
X_test = test.drop('id', axis = 1)

In [ ]:
hyperparams_xgbm = {
    'n_estimators': 1427, 
    'learning_rate': 0.045640623671124717, 
    'max_depth': 15, 
    'gamma': 2.1304843387906605, 
    'min_child_weight': 17, 
    'reg_alpha': 0.01490102698164227, 
    'reg_lambda': 0.053395147804257845, 
    'subsample': 0.8263124090290509, 
    'colsample_bytree': 0.8685865103401375, 
    'colsample_bylevel': 0.8566147867891335, 
    'colsample_bynode': 0.8021382585492708, 
    'grow_policy': 'lossguide'
}

In [ ]:
hyperparams_sgbm = {
     'n_estimators': 1494, 
     'learning_rate': 9.141062667777569e-05, 
     'max_depth': 15, 
     'min_samples_split': 10, 
     'min_samples_leaf': 5, 
     'subsample': 0.8850497055161204, 
     'max_features': 'log2'
}

In [ ]:
hyperparams_lr = {
    'penalty': 'l2', 
    'C': 0.3255323004350203, 
    'solver': 'liblinear', 
    'max_iter': 486, 
    'class_weight': 'balanced'
    }

In [ ]:
cat = CatBoostClassifier(depth = 11, random_seed = 123, eval_metric = 'AUC')
sgbm = GradientBoostingClassifier(**hyperparams_sgbm)
xgbm = XGBoostClassifier(**hyperparams_xgbm)
lr = LogisticRegression(**hyperparams_lr)

estimators = [('CatBoost', cat), ('Stochastic', sgbm), ('XGBoost', xgbm), ('Logistic', lr)]

vc = VotingClassifier(estimators = estimators, voting = 'soft', verbose = True)
vc.fit(X_train, y_train)

In [ ]:
y_test_prob = vc.predict_proba(X_test)[:, 1]

### Submission

In [ ]:
submission = pd.DataFrame({
    'id' : test['id'],
    'Response' : y_test_prob
})